In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv("Raw_Data.csv")

df.head()

/var/folders/bk/xdh674jn09g6ym3_3c5bpkhr0000gn/T/ipykernel_45870/2676799703.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Raw_Data.csv")


,Date,EL_Solar_BusBarn_Total_KW,Year,5,Day,Hour,Minute,DHI,DNI,GHI,...,Fill Flag,Surface Albedo,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm)
0,4/1/14 4:00,[-11059] No Good Data For Calculation,2014.0,4.0,1.0,4.0,30.0,0.0,0.0,0.0,...,0.0,0.157,7.8,0.849,271.4,59.41,-3.0,980.0,0.0,0.0
1,4/1/14 5:00,[-11059] No Good Data For Calculation,2014.0,4.0,1.0,5.0,30.0,0.0,0.0,0.0,...,0.0,0.157,7.3,0.621,272.9,59.59,-4.0,980.0,0.0,0.0
2,4/13/14 4:00,6.02E+30,2014.0,4.0,13.0,4.0,30.0,0.0,0.0,0.0,...,0.0,0.160,2.3,3.144,252.8,100.00,13.0,970.0,0.0,0.0
3,4/14/14 2:00,6.02E+30,2014.0,4.0,14.0,2.0,30.0,0.0,0.0,0.0,...,0.0,0.160,8.8,1.711,342.8,93.83,1.0,980.0,0.0,0.0
4,4/14/14 5:00,6.02E+30,2014.0,4.0,14.0,5.0,30.0,0.0,0.0,0.0,...,0.0,0.160,7.2,1.386,347.5,88.41,0.0,980.0,0.0,0.0


In [26]:
# I have a date and time in the coloumn Date where we have a data for each hour which is represented in each row, I want to 
# group the date in single date instead of having 24 rows for each date. I want to have a single row for each date and the
# data in the column EL_Solar_BusBarn_Total_KW should be summed up for each date before updating the data in EL_Solar_BusBarn_Total_KW
# column i want average of it but instead of dividing it by 24 i want to divide it by the number of rows for which have the non zero
# value in the column EL_Solar_BusBarn_Total_KW

df['Date'] = pd.to_datetime(df['Date'])
df['Only_Date'] = df['Date'].dt.date
print(df.head())
#[remove the row if the column EL_Solar_BusBarn_Total_KW has this value "[-11059] No Good Data For Calculation"
df = df[df['EL_Solar_BusBarn_Total_KW'] != '[-11059] No Good Data For Calculation'] 
# EL_Solar_BusBarn_Total_KW convert to float
df['EL_Solar_BusBarn_Total_KW'] = df['EL_Solar_BusBarn_Total_KW'].astype(float)

# values greater than 38 are considered as outliers and should replace with 0
df['EL_Solar_BusBarn_Total_KW'] = np.where(df['EL_Solar_BusBarn_Total_KW'] > 38, 0, df['EL_Solar_BusBarn_Total_KW'])

print(df.head())

summed_df = df.groupby('Only_Date')['EL_Solar_BusBarn_Total_KW'].sum().reset_index()
count_non_zero = df[df['EL_Solar_BusBarn_Total_KW'] != 0].groupby('Only_Date').size().reset_index(name='non_zero_count')
merged_df = pd.merge(summed_df, count_non_zero, on='Only_Date', how='left')
merged_df['Average_KW'] = merged_df['EL_Solar_BusBarn_Total_KW'] / merged_df['non_zero_count']

merged_df.head()
merged_df.to_csv('day-wise-data.csv', index=False)


                 Date  EL_Solar_BusBarn_Total_KW    Year    5   Day  Hour  \
2 2014-04-13 04:00:00                        0.0  2014.0  4.0  13.0   4.0   
3 2014-04-14 02:00:00                        0.0  2014.0  4.0  14.0   2.0   
4 2014-04-14 05:00:00                        0.0  2014.0  4.0  14.0   5.0   
5 2014-04-11 23:00:00                        0.0  2014.0  4.0  11.0  23.0   
6 2014-04-13 03:00:00                        0.0  2014.0  4.0  13.0   3.0   

   Minute  DHI  DNI  GHI  ...  Surface Albedo  Wind Speed  Precipitable Water  \
2    30.0  0.0  0.0  0.0  ...            0.16         2.3               3.144   
3    30.0  0.0  0.0  0.0  ...            0.16         8.8               1.711   
4    30.0  0.0  0.0  0.0  ...            0.16         7.2               1.386   
5    30.0  0.0  0.0  0.0  ...            0.16         3.8               1.668   
6    30.0  0.0  0.0  0.0  ...            0.16         3.5               3.271   

   Wind Direction  Relative Humidity  Temperature 

In [27]:
# I have a date and time in the coloumn Date where we have a data for each hour which is represented in each row, I want to 
# group the date in single date instead of having 24 rows for each date. I want to have a single row for each date and the
# data in the column GHI should be summed up for each date before updating the data in GHI
# column i want average of it but instead of dividing it by 24 i want to divide it by the number of rows for which have the non zero
# value in the column GHI

df['Date'] = pd.to_datetime(df['Date'])
df['Only_Date'] = df['Date'].dt.date

summed_df = df.groupby('Only_Date')['GHI'].sum().reset_index()
count_non_zero = df[df['GHI'] != 0].groupby('Only_Date').size().reset_index(name='non_zero_count')
merged_df = pd.merge(summed_df, count_non_zero, on='Only_Date', how='left')
merged_df['Average_GHI'] = merged_df['GHI'] / merged_df['non_zero_count']

merged_df.head()
merged_df.to_csv('day-wise-data-GHI.csv', index=False)


In [28]:
df = pd.read_csv("day-wise-data.csv")

df.head()

average_KW = df['Average_KW'].sum() / df['Average_KW'].count()
print("average_KW", average_KW)

average_GHI = df['Average_GHI'].sum() / df['Average_GHI'].count()
print("average_GHI", average_GHI)

area_m2 = 267.13 * 0.092903

print("Conversion Efficiency", average_KW / (average_GHI * area_m2) * 100, "%")


average_KW 8.547092046083204
average_GHI 358.14728588287255
Conversion Efficiency 0.09616218309145598 %


In [29]:
import pandas as pd

# Sample data loading step, replace this with your actual DataFrame
# df = pd.read_csv("your_data.csv")

# Convert 'Only_Date' to datetime and create a new column 'Month_Year' for grouping by month
df['Only_Date'] = pd.to_datetime(df['Only_Date'])
df['Month_Year'] = df['Only_Date'].dt.to_period('M')

# Sum of EL_Solar_BusBarn_Total_KW
summed_df = df.groupby('Month_Year')['EL_Solar_BusBarn_Total_KW'].sum().reset_index()

# Count of non-zero EL_Solar_BusBarn_Total_KW
count_non_zero = df.groupby('Month_Year')['EL_Solar_BusBarn_Total_KW'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count')

# Sum of GHI
sum_ghi = df.groupby('Month_Year')['GHI'].sum().reset_index()

# Count of non-zero GHI
count_non_zero_ghi = df.groupby('Month_Year')['GHI'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count_ghi')

# Merge the dataframes
merged_df = pd.merge(summed_df, count_non_zero, on='Month_Year', how='left')
merged_df = pd.merge(merged_df, sum_ghi, on='Month_Year', how='left')
merged_df = pd.merge(merged_df, count_non_zero_ghi, on='Month_Year', how='left')

# Calculate the averages
merged_df['Average_KW'] = merged_df['EL_Solar_BusBarn_Total_KW'] / merged_df['non_zero_count']
merged_df['Average_GHI'] = merged_df['GHI'] / merged_df['non_zero_count_ghi']

# Conversion efficiency calculations
area_m2 = 267.13 * 0.092903 # Conversion from sq ft to m2
merged_df['Conversion_Efficiency'] = (merged_df['Average_KW'] / (merged_df['Average_GHI'] * area_m2)) * 100

# Display the head of the final DataFrame
print(merged_df.head())

# Save the final DataFrame to a CSV file
merged_df.to_csv('month-wise-data.csv', index=False)


  Month_Year  EL_Solar_BusBarn_Total_KW  non_zero_count     GHI  \
0    2011-05                2436.461646              18  103992   
1    2011-06                4434.415868              30  172306   
2    2011-07                5524.866557              31  200736   
3    2011-08                5700.346898              31  191865   
4    2011-09                3932.046562              30  135983   

   non_zero_count_ghi  Average_KW  Average_GHI  Conversion_Efficiency  
0                  18  135.358980  5777.333333               0.094408  
1                  30  147.813862  5743.533333               0.103701  
2                  31  178.221502  6475.354839               0.110903  
3                  31  183.882158  6189.193548               0.119716  
4                  30  131.068219  4532.766667               0.116515  


/var/folders/bk/xdh674jn09g6ym3_3c5bpkhr0000gn/T/ipykernel_45870/3927180844.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Only_Date'] = pd.to_datetime(df['Only_Date'])


In [30]:
import pandas as pd

# Sample data loading step, replace this with your actual DataFrame
# df = pd.read_csv("your_data.csv")

# Convert 'Only_Date' to datetime and create a new column 'Year' for grouping by year
df['Only_Date'] = pd.to_datetime(df['Only_Date'])
df['Year'] = df['Only_Date'].dt.year

# Sum of EL_Solar_BusBarn_Total_KW
summed_df = df.groupby('Year')['EL_Solar_BusBarn_Total_KW'].sum().reset_index()

# Count of non-zero EL_Solar_BusBarn_Total_KW
count_non_zero = df.groupby('Year')['EL_Solar_BusBarn_Total_KW'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count')

# Sum of GHI
sum_ghi = df.groupby('Year')['GHI'].sum().reset_index()

# Count of non-zero GHI
count_non_zero_ghi = df.groupby('Year')['GHI'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count_ghi')

# Merge the dataframes
merged_df = pd.merge(summed_df, count_non_zero, on='Year', how='left')
merged_df = pd.merge(merged_df, sum_ghi, on='Year', how='left')
merged_df = pd.merge(merged_df, count_non_zero_ghi, on='Year', how='left')

# Calculate the averages
merged_df['Average_KW'] = merged_df['EL_Solar_BusBarn_Total_KW'] / merged_df['non_zero_count']
merged_df['Average_GHI'] = merged_df['GHI'] / merged_df['non_zero_count_ghi']

# Conversion efficiency calculations
area_m2 = 267.13 * 0.092903 # Conversion from sq ft to m2
merged_df['Conversion_Efficiency'] = (merged_df['Average_KW'] / (merged_df['Average_GHI'] * area_m2)) * 100

# Display the head of the final DataFrame
print(merged_df.head())

# Save the final DataFrame to a CSV file
merged_df.to_csv('year-wise-data.csv', index=False)


   Year  EL_Solar_BusBarn_Total_KW  non_zero_count      GHI  \
0  2011               28677.930803             232  1025625   
1  2012               45501.532046             366  1612956   
2  2013               40906.421029             364  1523598   
3  2014               39292.551871             364  1482753   
4  2015               40671.415153             362  1517825   

   non_zero_count_ghi  Average_KW  Average_GHI  Conversion_Efficiency  
0                 232  123.611771  4420.797414               0.112670  
1                 366  124.321126  4406.983607               0.113671  
2                 365  112.380278  4174.241096               0.108483  
3                 365  107.946571  4062.336986               0.107073  
4                 365  112.351976  4158.424658               0.108868  


In [31]:
df = pd.read_csv("year-wise-data.csv")

sum_except_last = df["Conversion_Efficiency"].sum()

print("Average Conversion Efficiency", sum_except_last)

Average Conversion Efficiency 1.2099298496133417


In [38]:
df = pd.read_csv("month-wise-data.csv")

df.head()

,Month_Year,EL_Solar_BusBarn_Total_KW,non_zero_count,GHI,non_zero_count_ghi,Average_KW,Average_GHI,Conversion_Efficiency
0,2011-05,2436.461646,18,103992,18,135.358980,5777.333333,0.094408
1,2011-06,4434.415868,30,172306,30,147.813862,5743.533333,0.103701
2,2011-07,5524.866557,31,200736,31,178.221502,6475.354839,0.110903
3,2011-08,5700.346898,31,191865,31,183.882158,6189.193548,0.119716
4,2011-09,3932.046562,30,135983,30,131.068219,4532.766667,0.116515


In [37]:
import pandas as pd

# Sample data loading step, replace this with your actual DataFrame
df = pd.read_csv("day-wise-data.csv")

# Convert 'Only_Date' to datetime
df['Only_Date'] = pd.to_datetime(df['Date']).dt.date

# Sum of EL_Solar_BusBarn_Total_KW
summed_df = df.groupby('Only_Date')['EL_Solar_BusBarn_Total_KW'].sum().reset_index()

# Count of non-zero EL_Solar_BusBarn_Total_KW
count_non_zero = df.groupby('Only_Date')['EL_Solar_BusBarn_Total_KW'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count')

# Sum of GHI
sum_ghi = df.groupby('Only_Date')['GHI'].sum().reset_index()

# Count of non-zero GHI
count_non_zero_ghi = df.groupby('Only_Date')['GHI'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count_ghi')

# Merge the dataframes
merged_df = pd.merge(summed_df, count_non_zero, on='Only_Date', how='left')
merged_df = pd.merge(merged_df, sum_ghi, on='Only_Date', how='left')
merged_df = pd.merge(merged_df, count_non_zero_ghi, on='Only_Date', how='left')

# Calculate the averages
merged_df['Average_KW'] = merged_df['EL_Solar_BusBarn_Total_KW'] / merged_df['non_zero_count']
merged_df['Average_GHI'] = merged_df['GHI'] / merged_df['non_zero_count_ghi']

# Conversion Efficiency calculations
area_m2 = 267.13 * 0.092903  # Conversion from sq ft to m2
merged_df['Conversion_Efficiency'] = (merged_df['Average_KW'] / (merged_df['Average_GHI'] * area_m2)) * 100

# Display the head of the final DataFrame
print(merged_df.head())

# Save the final DataFrame to a CSV file
merged_df.to_csv('day-wise-data.csv', index=False)


KeyError: 'Date'

In [33]:
import pandas as pd

# Sample data loading step, replace this with your actual DataFrame
df = pd.read_csv("Raw_Data.csv")

# Convert 'Date' to datetime and extract date and hour
df['Datetime'] = pd.to_datetime(df['Date'])
df['Only_Date'] = df['Datetime'].dt.date
df['Hour'] = df['Datetime'].dt.hour
#[remove the row if the column EL_Solar_BusBarn_Total_KW has this value "[-11059] No Good Data For Calculation"
df = df[df['EL_Solar_BusBarn_Total_KW'] != '[-11059] No Good Data For Calculation'] 
# EL_Solar_BusBarn_Total_KW convert to float
df['EL_Solar_BusBarn_Total_KW'] = df['EL_Solar_BusBarn_Total_KW'].astype(float)

# values greater than 38 are considered as outliers and should replace with 0
df['EL_Solar_BusBarn_Total_KW'] = np.where(df['EL_Solar_BusBarn_Total_KW'] > 38, 0, df['EL_Solar_BusBarn_Total_KW'])

print(df.head())


/var/folders/bk/xdh674jn09g6ym3_3c5bpkhr0000gn/T/ipykernel_45870/21405734.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Raw_Data.csv")
/var/folders/bk/xdh674jn09g6ym3_3c5bpkhr0000gn/T/ipykernel_45870/21405734.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Datetime'] = pd.to_datetime(df['Date'])


            Date  EL_Solar_BusBarn_Total_KW    Year    5   Day  Hour  Minute  \
2   4/13/14 4:00                        0.0  2014.0  4.0  13.0     4    30.0   
3   4/14/14 2:00                        0.0  2014.0  4.0  14.0     2    30.0   
4   4/14/14 5:00                        0.0  2014.0  4.0  14.0     5    30.0   
5  4/11/14 23:00                        0.0  2014.0  4.0  11.0    23    30.0   
6   4/13/14 3:00                        0.0  2014.0  4.0  13.0     3    30.0   

   DHI  DNI  GHI  ...  Wind Speed  Precipitable Water  Wind Direction  \
2  0.0  0.0  0.0  ...         2.3               3.144           252.8   
3  0.0  0.0  0.0  ...         8.8               1.711           342.8   
4  0.0  0.0  0.0  ...         7.2               1.386           347.5   
5  0.0  0.0  0.0  ...         3.8               1.668           195.4   
6  0.0  0.0  0.0  ...         3.5               3.271           239.2   

   Relative Humidity  Temperature  Pressure  \
2             100.00         13.0

In [34]:
# Group by date and hour, then sum EL_Solar_BusBarn_Total_KW
summed_df = df.groupby(['Only_Date', 'Hour'])['EL_Solar_BusBarn_Total_KW'].sum().reset_index()

# Group by date and hour, then count non-zero EL_Solar_BusBarn_Total_KW
count_non_zero = df.groupby(['Only_Date', 'Hour'])['EL_Solar_BusBarn_Total_KW'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count')

# Group by date and hour, then sum GHI
sum_ghi = df.groupby(['Only_Date', 'Hour'])['GHI'].sum().reset_index()

# Group by date and hour, then count non-zero GHI
count_non_zero_ghi = df.groupby(['Only_Date', 'Hour'])['GHI'].apply(lambda x: (x != 0).sum()).reset_index(name='non_zero_count_ghi')

# Merge the dataframes
merged_df = pd.merge(summed_df, count_non_zero, on=['Only_Date', 'Hour'], how='left')
merged_df = pd.merge(merged_df, sum_ghi, on=['Only_Date', 'Hour'], how='left')
merged_df = pd.merge(merged_df, count_non_zero_ghi, on=['Only_Date', 'Hour'], how='left')

# Calculate the averages
merged_df['Average_KW'] = merged_df['EL_Solar_BusBarn_Total_KW'] / merged_df['non_zero_count']
merged_df['Average_GHI'] = merged_df['GHI'] / merged_df['non_zero_count_ghi']

# Conversion Efficiency calculations
area_m2 = 267.13 * 0.092903  # Conversion from sq ft to m2
merged_df['Conversion_Efficiency'] = (merged_df['Average_KW'] / (merged_df['Average_GHI'] * area_m2)) * 100

# Display the head of the final DataFrame
print(merged_df.head())

# Save the final DataFrame to a new CSV file
merged_df.to_csv('hour-wise-conversion-efficiency.csv', index=False)

    Only_Date  Hour  EL_Solar_BusBarn_Total_KW  non_zero_count  GHI  \
0  2011-05-14     0                        0.0               0  0.0   
1  2011-05-14     1                        0.0               0  0.0   
2  2011-05-14     2                        0.0               0  0.0   
3  2011-05-14     3                        0.0               0  0.0   
4  2011-05-14     4                        0.0               0  0.0   

   non_zero_count_ghi  Average_KW  Average_GHI  Conversion_Efficiency  
0                   0         NaN          NaN                    NaN  
1                   0         NaN          NaN                    NaN  
2                   0         NaN          NaN                    NaN  
3                   0         NaN          NaN                    NaN  
4                   0         NaN          NaN                    NaN  
